In [ ]:
import cv2
import mediapipe as mp
import time
import pygame
import random

# Initialize Pygame
pygame.init()
pygame.font.init()

# Pygame window setup
WINDOW_WIDTH = 800
WINDOW_HEIGHT = 600
window = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
pygame.display.set_caption("Hand Games")
font = pygame.font.Font(None, 74)
small_font = pygame.font.Font(None, 54)

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
YELLOW = (255, 255, 0)

# Initialize MediaPipe Hands module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils
cap = cv2.VideoCapture(0)

def reset_rps_state():
    return {
        "game_active": True,
        "game_start_time": time.time(),
        "game_state": "Get Ready",
        "computer_choice": None,
        "result": None,
        "current_index": -1,
        "last_change_time": time.time(),
        "show_replay": False,
        "waiting_for_move": False,
        "move_start_time": None,
        "no_hands_detected": False,
        "no_hands_start_time": None
    }

def reset_cricket_state():
    return {
        "game_active": True,
        "game_start_time": time.time(),
        "game_state": "Get Ready",
        "computer_choice": None,
        "player_score": 0,
        "computer_score": 0,
        "current_index": -1,
        "last_change_time": time.time(),
        "show_replay": False,
        "waiting_for_move": False,
        "move_start_time": None,
        "no_hands_detected": False,
        "no_hands_start_time": None,
        "innings": 1,
        "target": None,
        "result": None
    }

# Game states
game_selection = True
current_game = None
game_vars = None
rps_sequence = ["Rock", "Paper", "Scissors", "Shoot"]
cricket_sequence = ["Ready", "Set", "Show"]
CHANGE_DELAY = 0.7
RESULT_DISPLAY_TIME = 2.5
GET_READY_DELAY = 1.0
NO_HANDS_DELAY = 1.0
MOVE_TIMEOUT = 3.0

def get_finger_count(hand_landmarks):
    finger_tips = [
        mp_hands.HandLandmark.INDEX_FINGER_TIP,
        mp_hands.HandLandmark.MIDDLE_FINGER_TIP,
        mp_hands.HandLandmark.RING_FINGER_TIP,
        mp_hands.HandLandmark.PINKY_TIP
    ]
    finger_mcps = [
        mp_hands.HandLandmark.INDEX_FINGER_MCP,
        mp_hands.HandLandmark.MIDDLE_FINGER_MCP,
        mp_hands.HandLandmark.RING_FINGER_MCP,
        mp_hands.HandLandmark.PINKY_MCP
    ]
    
    count = 0
    for tip, mcp in zip(finger_tips, finger_mcps):
        if hand_landmarks.landmark[tip].y < hand_landmarks.landmark[mcp].y:
            count += 1
    
    # Special case for thumb
    if hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x < hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP].x:
        count += 1
    
    return count

def is_fist(hand_landmarks):
    fingertips = [mp_hands.HandLandmark.INDEX_FINGER_TIP, mp_hands.HandLandmark.MIDDLE_FINGER_TIP, 
                  mp_hands.HandLandmark.RING_FINGER_TIP, mp_hands.HandLandmark.PINKY_TIP]
    pips = [mp_hands.HandLandmark.INDEX_FINGER_PIP, mp_hands.HandLandmark.MIDDLE_FINGER_PIP, 
            mp_hands.HandLandmark.RING_FINGER_PIP, mp_hands.HandLandmark.PINKY_PIP]
    
    for fingertip, pip in zip(fingertips, pips):
        if hand_landmarks.landmark[fingertip].y < hand_landmarks.landmark[pip].y:
            return False
    return True

def get_rps_move(hand_landmarks):
    if is_fist(hand_landmarks):
        return "Rock"
    elif get_finger_count(hand_landmarks) == 5:
        return "Paper"
    else:
        return "Scissors"

def determine_rps_winner(player_move, computer_move):
    if player_move == computer_move:
        return "Ughh tie"
    elif (player_move == "Rock" and computer_move == "Scissors") or \
         (player_move == "Paper" and computer_move == "Rock") or \
         (player_move == "Scissors" and computer_move == "Paper"):
        return "boringgg you win"
    else:
        return "Ya lose lollll!"

def process_cricket_move(player_move, computer_move, game_vars):
    if player_move == computer_move:
        if game_vars["innings"] == 1:
            game_vars["innings"] = 2
            game_vars["target"] = game_vars["player_score"] + 1
            return f"OUT! Target: {game_vars['target']}"
        else:
            game_vars["result"] = "Computer wins!" if game_vars["computer_score"] > game_vars["player_score"] else "You win!"
            return "OUT! Game Over!"
    else:
        if game_vars["innings"] == 1:
            game_vars["player_score"] += player_move
        else:
            game_vars["computer_score"] += computer_move
            if game_vars["computer_score"] > game_vars["target"]:
                game_vars["result"] = "Computer wins!"
                return "Game Over!"
        return f"Runs: {player_move}"

# Main game loop
while True:
    # Handle Pygame events
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            cap.release()
            cv2.destroyAllWindows()
            exit()
        elif event.type == pygame.KEYDOWN:
            if game_selection:
                if event.key == pygame.K_r:
                    current_game = "rps"
                    game_vars = reset_rps_state()
                    game_selection = False
                elif event.key == pygame.K_c:
                    current_game = "cricket"
                    game_vars = reset_cricket_state()
                    game_selection = False
            elif event.key == pygame.K_SPACE and game_vars["show_replay"]:
                game_selection = True
                game_vars = None
            elif event.key == pygame.K_q:
                pygame.quit()
                cap.release()
                cv2.destroyAllWindows()
                exit()

    # Clear the window
    window.fill(BLACK)

    if game_selection:
        # Show game selection menu
        title_text = font.render("Choose Your Game", True, WHITE)
        title_rect = title_text.get_rect(center=(WINDOW_WIDTH/2, WINDOW_HEIGHT/3))
        window.blit(title_text, title_rect)

        rps_text = small_font.render("Press R for Rock Paper Scissors", True, WHITE)
        rps_rect = rps_text.get_rect(center=(WINDOW_WIDTH/2, WINDOW_HEIGHT/2))
        window.blit(rps_text, rps_rect)

        cricket_text = small_font.render("Press C for Hand Cricket", True, WHITE)
        cricket_rect = cricket_text.get_rect(center=(WINDOW_WIDTH/2, 2*WINDOW_HEIGHT/3))
        window.blit(cricket_text, cricket_rect)

    elif not game_vars["show_replay"]:
        current_time = time.time()
        sequence = rps_sequence if current_game == "rps" else cricket_sequence

        # Handle "Get Ready" state
        if game_vars["game_state"] == "Get Ready":
            if current_time - game_vars["last_change_time"] >= GET_READY_DELAY:
                game_vars["current_index"] = 0
                game_vars["game_state"] = sequence[0]
                game_vars["last_change_time"] = current_time

        # Handle main game sequence
        elif current_time - game_vars["last_change_time"] >= CHANGE_DELAY:
            if game_vars["current_index"] < len(sequence) - 1:
                game_vars["current_index"] += 1
                game_vars["game_state"] = sequence[game_vars["current_index"]]
                game_vars["last_change_time"] = current_time
                
                if game_vars["game_state"] in ["Shoot", "Show"]:
                    if current_game == "rps":
                        game_vars["computer_choice"] = random.choice(["Rock", "Paper", "Scissors"])
                    else:
                        game_vars["computer_choice"] = random.randint(0, 6)
                    game_vars["waiting_for_move"] = True
                    game_vars["move_start_time"] = current_time
                    game_vars["no_hands_start_time"] = current_time

        # Read camera frame and process hand detection
        ret, frame = cap.read()
        if ret:
            frame = cv2.flip(frame, 1)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(frame_rgb)

            if game_vars["waiting_for_move"] and not game_vars.get("result"):
                if results.multi_hand_landmarks:
                    game_vars["no_hands_detected"] = False
                    for hand_landmarks in results.multi_hand_landmarks:
                        mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                        if current_game == "rps":
                            player_move = get_rps_move(hand_landmarks)
                            game_vars["result"] = determine_rps_winner(player_move, game_vars["computer_choice"])
                            game_vars["player_move"] = player_move
                        else:
                            player_move = get_finger_count(hand_landmarks)
                            game_vars["player_move"] = player_move
                            game_vars["result"] = process_cricket_move(player_move, game_vars["computer_choice"], game_vars)

                elif current_time - game_vars["no_hands_start_time"] >= NO_HANDS_DELAY:
                    game_vars["no_hands_detected"] = True
                    
                if current_time - game_vars["move_start_time"] >= MOVE_TIMEOUT:
                    game_vars["result"] = "No move detected - Try again!"
                    game_vars["player_move"] = "No move"

            elif results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            cv2.imshow('Hand Detection', frame)

        # Draw game state
        text = font.render(game_vars["game_state"], True, WHITE)
        text_rect = text.get_rect(center=(WINDOW_WIDTH/2, WINDOW_HEIGHT/3))
        window.blit(text, text_rect)

        # Draw game-specific information
        if current_game == "cricket":
            score_text = small_font.render(f"Your Score: {game_vars['player_score']} | Computer Score: {game_vars['computer_score']}", True, WHITE)
            score_rect = score_text.get_rect(center=(WINDOW_WIDTH/2, 50))
            window.blit(score_text, score_rect)

            if game_vars["target"]:
                target_text = small_font.render(f"Target: {game_vars['target']}", True, WHITE)
                target_rect = target_text.get_rect(center=(WINDOW_WIDTH/2, 100))
                window.blit(target_text, target_rect)

        # Draw "No hands" message if needed
        if game_vars["waiting_for_move"] and game_vars["no_hands_detected"] and not game_vars["result"]:
            no_hands_text = small_font.render("Show your hand!", True, YELLOW)
            no_hands_rect = no_hands_text.get_rect(center=(WINDOW_WIDTH/2, WINDOW_HEIGHT/2))
            window.blit(no_hands_text, no_hands_rect)

        # Draw moves and result
        if game_vars["computer_choice"] is not None and game_vars["result"]:
            if current_game == "rps":
                choice_text = small_font.render(f"Computer chose: {game_vars['computer_choice']}", True, WHITE)
            else:
                choice_text = small_font.render(f"Computer showed: {game_vars['computer_choice']}", True, WHITE)
            choice_rect = choice_text.get_rect(center=(WINDOW_WIDTH/2, WINDOW_HEIGHT/2 - 40))
            window.blit(choice_text, choice_rect)

            player_text = small_font.render(f"You showed: {game_vars['player_move']}", True, WHITE)
            player_rect = player_text.get_rect(center=(WINDOW_WIDTH/2, WINDOW_HEIGHT/2 + 40))
            window.blit(player_text, player_rect)

            result_text = font.render(game_vars["result"], True, WHITE)
            result_rect = result_text.get_rect(center=(WINDOW_WIDTH/2, 2*WINDOW_HEIGHT/3))
            window.blit(result_text, result_rect)

            if current_time - game_vars["move_start_time"] >= RESULT_DISPLAY_TIME:
                if current_game == "rps" or (current_game == "cricket" and game_vars.get("result") and "Game Over" in game_vars["result"]):
                    game_vars["show_replay"] = True
                else:
                    game_vars["game_state"] = "Get Ready"
                    game_vars["current_index"] = -1
                    game_vars["last_change_time"] = current_time
                    game_vars["computer_choice"] = None
                    game_vars["result"] = None
                    game_vars["waiting_for_move"] = False
                    game_vars["no_hands_detected"] = False

    else:
        # Show replay screen
        title_text = font.render("Play Again?", True, WHITE)
        title_rect = title_text.get_rect(center=(WINDOW_WIDTH/2, WINDOW_HEIGHT/3))
        window.blit(title_text, title_rect)

        instruction_text = small_font.render("Press SPACE to play again", True, WHITE)
        instruction_rect = instruction_text.get_rect(center=(WINDOW_WIDTH/2, WINDOW_HEIGHT/2))
        window.blit(instruction_text, instruction_rect)

        quit_text = small_font.render("Press Q to quit", True, WHITE)
        quit_rect = quit_text.get_rect(center=(WINDOW_WIDTH/2, 2*WINDOW_HEIGHT/3))
        window.blit(quit_text, quit_rect)

    # Update the display
    pygame.display.flip()

    # Exit condition
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()
pygame.quit()